In [80]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import numpy
import re
import string

In [81]:
taipei_req = Request('https://weather.com/weather/hourbyhour/l/74fa72e237fc1737b20f120d74160a01d04e634ef78f96b3e76e9047acac201a', headers={'User-Agent': 'Mozilla/5.0'})
nangang_req = Request('https://weather.com/weather/hourbyhour/l/fd7431ad84264a82af4b40edf4a2f39f81f53d83d2de976d8c33ddb01fcdef76', headers={'User-Agent': 'Mozilla/5.0'})
banqiao_req = Request('https://weather.com/weather/hourbyhour/l/0e5e6557b2aed2af45d4b5eb0ea0a8a77778551f6108d1ec13dd384f4daa3658', headers={'User-Agent': 'Mozilla/5.0'})
req = [taipei_req,nangang_req,banqiao_req]

In [82]:
#取日期
def get_date(soup):
    date = []
    day = []
    for link in soup.findAll('h3',attrs={'class': re.compile("HourlyForecast--longDate--3khKr")}):
        s = link.get_text().replace('January', '2021/1/')
        s = s.replace(" ","")
        buffer = s.split(",")
        day.append(buffer[0])
        date.append(buffer[1])
    #     print(date)
    return date,day
    # print(len(day))

In [83]:
#取時間
def get_time_hr(soup):
    time_hr = []
    for link in soup.findAll('h2',attrs={'data-testid': re.compile("daypartName")}):
        if link.get_text()[len(link.get_text())-2] == 'p':
            time = int(link.get_text()[:len(link.get_text())-3]) + 13
        else:
            time = int(link.get_text()[:len(link.get_text())-3]) + 1

        if time > 24:
            time = time - 24

        if link.get_text() == "12 am":
            time = 1
        elif link.get_text() == "12 pm":
            time = 13

        time_hr.append(time)
    return time_hr
    #     print(link.get_text())
    #     print(time)
#     print(time_hr)

In [84]:
#取氣溫
def get_temperature(soup):
    temperature = []
    for link in soup.findAll("span",class_ = "DetailsTable--value--1F3Ze",attrs={'data-testid': re.compile("TemperatureValue")}):
        t = int(link.get_text()[:len(link.get_text())-1])
        t = (t -32) * 5 / 9
        temperature.append(round(t,1))
    #     print(round(t,1))
#     print(len(temperature))
    return temperature

In [85]:
#取下雨情況
def get_rainy_txt(soup):
    rainy_txt = []
    for link in soup.findAll('span',attrs={'class': re.compile("DetailsSummary--extendedData--aaFeV")}):
        rainy_txt.append(link.get_text())
    #     print(link.get_text())
#     print(len(rainy_txt))
    return rainy_txt

In [86]:
#取降雨機率
def get_rainy_percent(soup):
    rainy_percent = []
    for link in soup.findAll("div",attrs={'class': re.compile("DetailsSummary--precip--2ARnx")}):
    #     print(link)
        for link2 in link.findAll('span',attrs={'data-testid': re.compile("PercentageValue")}):
            rainy_percent.append(link2.get_text()[:len(link.get_text())-1])
    #         print(link2.get_text())
#     print(len(rainy_percent))
    return rainy_percent

In [87]:
#取風向大小
def get_wind(soup):
    wind = []
    for link in soup.findAll("div",attrs={'data-testid': re.compile("wind")}):
    #     print(link)
        for link2 in link.findAll('span',attrs={'data-testid': re.compile("Wind")}):
            wind.append(link2.get_text())
    #         print(link2.get_text())
#     print(len(wind))
    return wind

In [88]:
#取濕度
def get_humidity(soup):
    humidity = []
    for link in soup.findAll("span",class_ = "DetailsTable--value--1F3Ze",attrs={'data-testid': re.compile("PercentageValue")}):
        humidity.append(link.get_text()[:len(link.get_text())-1])
    #     print(type(link.get_text()))
#     print(len(humidity))
    return humidity

In [89]:
name = ["taipei", "nangang", "banqiao"]
for r in range(len(req)):
    webpage = urlopen(req[r]).read()
    soup = BeautifulSoup(webpage)

    date,day = get_date(soup)
    time_hr = get_time_hr(soup)
    temperature = get_temperature(soup)
    rainy_txt = get_rainy_txt(soup)
    rainy_percent = get_rainy_percent(soup)
    wind = get_wind(soup)
    humidity = get_humidity(soup)

    data = []
    buffer = ["day", "date", "time_hr", "temperature", "rainy_txt", "rainy_percent", "wind", "humidity"]
    data.append(buffer)

    num = 0
    for i in range(len(time_hr)):
        buffer = []
        buffer.append(day[num])
        buffer.append(date[num])
        buffer.append(time_hr[i])
        buffer.append(temperature[i])
        buffer.append(rainy_txt[i])
        buffer.append(rainy_percent[i])
        buffer.append(wind[i])
        buffer.append(humidity[i])
        data.append(buffer)
    #     print(time_hr[i])
    #     print("num",num)
        if time_hr[i] == 24:
            num = num + 1

    numpy.savetxt(name[r] + "_future_48hr_weather.csv", data, fmt='%s',delimiter=",")

In [90]:
##星期 day 3
#日期 date 3
#時間 time_hr 48
#氣溫 temperature 48
#下雨情況 rainy_txt 48
#降雨機率 rainy_percent 48
#風向大小 wind 48
#濕度 humidity 48
#taipei
#nangang
#banqiao

In [91]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None

In [92]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [93]:
#train data
data= pd.read_csv('./mrt_weather.csv')

In [94]:
station2label={'南港':1, '台北車站':2, '板橋':3}
for i in range(len(data)):
    data['進站'][i]=station2label[data['進站'][i]]

In [95]:
X=data[['進站', '時段', '氣溫(℃)', '相對溼度(%)' ]]
Y=data['人次']
# print(X)
# print(Y)

In [96]:
X_train, X_test, Y_train, Y_test= train_test_split(X, Y, test_size=0.25, random_state=33)

In [97]:
test = RandomForestClassifier()
test.fit(X_train, Y_train)
predict=test.predict(X_test)

In [98]:
nangang_data= pd.read_csv('./nangang_future_48hr_weather.csv')
print(nangang_data)

         day       date  time_hr  temperature      rainy_txt  rainy_percent  \
0   Thursday  2021/1/14       16         20.6   Mostly Sunny              0   
1   Thursday  2021/1/14       17         19.4   Mostly Sunny              0   
2   Thursday  2021/1/14       18         17.8          Sunny              0   
3   Thursday  2021/1/14       19         16.1          Clear              0   
4   Thursday  2021/1/14       20         15.0          Clear              0   
5   Thursday  2021/1/14       21         13.9          Clear              0   
6   Thursday  2021/1/14       22         13.3          Clear              5   
7   Thursday  2021/1/14       23         13.3          Clear              5   
8   Thursday  2021/1/14       24         12.8          Clear              5   
9     Friday  2021/1/15        1         12.2          Clear              5   
10    Friday  2021/1/15        2         12.2          Clear              5   
11    Friday  2021/1/15        3         11.7       

In [99]:
list=[1 for n in range(48)]
nang_test=nangang_data[['time_hr', 'temperature', 'humidity']]
nang_test.insert(0 ,'station', list, True)

In [100]:
nang_RandomForest_predict=test.predict(nang_test)

In [101]:
from sklearn import tree

In [102]:
clf = tree.DecisionTreeClassifier()

In [103]:
clf.fit(X_train, Y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [104]:
nang_Tree_predict=clf.predict(nang_test)

In [105]:
label2station={1:'南港',2:'台北車站', 3:'板橋'}

station=[]
for i in nang_test['station']:
    station.append(str(label2station[i]))
    
t=[]
for i in nang_test['time_hr']:
    t.append(i)


In [106]:
from sklearn import neighbors

In [107]:
knn_model = neighbors.KNeighborsClassifier()

In [108]:
knn_clf=knn_model.fit(X_train, Y_train)

In [109]:
nang_Knn_predict=knn_clf.predict(nang_test)

In [110]:
output_test={'日期': nangang_data['date'].tolist(), '站名': station, '時段': t, '預測人數': nang_Tree_predict.tolist()}
df = pd.DataFrame(output_test)
df.to_csv('test_Tree.csv',index=False)


In [111]:
data= pd.read_csv('./test_Tree.csv')
print(data)

           日期  站名  時段  預測人數
0   2021/1/14  南港  16   896
1   2021/1/14  南港  17   591
2   2021/1/14  南港  18  1086
3   2021/1/14  南港  19   967
4   2021/1/14  南港  20   518
5   2021/1/14  南港  21   755
6   2021/1/14  南港  22   789
7   2021/1/14  南港  23   673
8   2021/1/14  南港  24   279
9   2021/1/15  南港   1    88
10  2021/1/15  南港   2     2
11  2021/1/15  南港   3     2
12  2021/1/15  南港   4     2
13  2021/1/15  南港   5     0
14  2021/1/15  南港   6     0
15  2021/1/15  南港   7    87
16  2021/1/15  南港   8   244
17  2021/1/15  南港   9  2221
18  2021/1/15  南港  10   578
19  2021/1/15  南港  11   551
20  2021/1/15  南港  12   629
21  2021/1/15  南港  13   580
22  2021/1/15  南港  14   580
23  2021/1/15  南港  15   596
24  2021/1/15  南港  16   580
25  2021/1/15  南港  17   702
26  2021/1/15  南港  18  2403
27  2021/1/15  南港  19  1065
28  2021/1/15  南港  20  1452
29  2021/1/15  南港  21   870
30  2021/1/15  南港  22   266
31  2021/1/15  南港  23   266
32  2021/1/15  南港  24   266
33  2021/1/16  南港   1    94
34  2021/1/16  南港   

In [112]:
output_test={'日期': nangang_data['date'].tolist(), '站名': station, '時段': t, '預測人數': nang_Knn_predict.tolist()}
df = pd.DataFrame(output_test)
df.to_csv('test_KNN.csv',index=False)


In [113]:
data= pd.read_csv('./test_KNN.csv')
print(data)

           日期  站名  時段  預測人數
0   2021/1/14  南港  16   896
1   2021/1/14  南港  17   916
2   2021/1/14  南港  18   951
3   2021/1/14  南港  19  1212
4   2021/1/14  南港  20   779
5   2021/1/14  南港  21   575
6   2021/1/14  南港  22   626
7   2021/1/14  南港  23   673
8   2021/1/14  南港  24   196
9   2021/1/15  南港   1     2
10  2021/1/15  南港   2     2
11  2021/1/15  南港   3     2
12  2021/1/15  南港   4     0
13  2021/1/15  南港   5     0
14  2021/1/15  南港   6     0
15  2021/1/15  南港   7     0
16  2021/1/15  南港   8     0
17  2021/1/15  南港   9  2221
18  2021/1/15  南港  10   497
19  2021/1/15  南港  11   263
20  2021/1/15  南港  12   289
21  2021/1/15  南港  13   340
22  2021/1/15  南港  14   528
23  2021/1/15  南港  15   543
24  2021/1/15  南港  16   543
25  2021/1/15  南港  17   580
26  2021/1/15  南港  18  1539
27  2021/1/15  南港  19   464
28  2021/1/15  南港  20   288
29  2021/1/15  南港  21  1202
30  2021/1/15  南港  22   627
31  2021/1/15  南港  23   279
32  2021/1/15  南港  24   266
33  2021/1/16  南港   1     2
34  2021/1/16  南港   

In [114]:
output_test={'日期': nangang_data['date'].tolist(), '站名': station, '時段': t, '預測人數': nang_RandomForest_predict.tolist()}
df = pd.DataFrame(output_test)
df.to_csv('test_RandomForest.csv',index=False)


In [115]:
data= pd.read_csv('./test_RandomForest.csv')
print(data)

           日期  站名  時段  預測人數
0   2021/1/14  南港  16   959
1   2021/1/14  南港  17   916
2   2021/1/14  南港  18  1065
3   2021/1/14  南港  19  1388
4   2021/1/14  南港  20   967
5   2021/1/14  南港  21   755
6   2021/1/14  南港  22   626
7   2021/1/14  南港  23   673
8   2021/1/14  南港  24   376
9   2021/1/15  南港   1    64
10  2021/1/15  南港   2     2
11  2021/1/15  南港   3     2
12  2021/1/15  南港   4     2
13  2021/1/15  南港   5     0
14  2021/1/15  南港   6     0
15  2021/1/15  南港   7    87
16  2021/1/15  南港   8   184
17  2021/1/15  南港   9   566
18  2021/1/15  南港  10   497
19  2021/1/15  南港  11   551
20  2021/1/15  南港  12   555
21  2021/1/15  南港  13   528
22  2021/1/15  南港  14   540
23  2021/1/15  南港  15   596
24  2021/1/15  南港  16   580
25  2021/1/15  南港  17   890
26  2021/1/15  南港  18   410
27  2021/1/15  南港  19  1877
28  2021/1/15  南港  20  1295
29  2021/1/15  南港  21  1202
30  2021/1/15  南港  22   666
31  2021/1/15  南港  23   873
32  2021/1/15  南港  24   266
33  2021/1/16  南港   1    94
34  2021/1/16  南港   